In [ ]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython
import sklearn
import seaborn as sns
from sklearn.utils import shuffle

%load_ext tensorboard

In [ ]:
#This notebook use ESC-50 datasets on Kaggle Kernel, follow the step on README for further instructions.
CSV_ESC = "../input/environmental-sound-classification-50/esc50.csv"  # path of csv file
DATA_ESC = "../input/environmental-sound-classification-50/audio/audio/16000/" # path to folder containing audio files

CSV_URBAN = "../input/urbansound8k/UrbanSound8K.csv"
DATA_URBAN = "../input/urbansound8k/fold"
# Reading the CSV File
df1 = pd.read_csv(CSV_ESC)
df2 = pd.read_csv(CSV_URBAN)

In [ ]:
# Coba liat kode diatas, pertama ganti kolom kolomnya biar mirip
df_esc = df1.drop(columns=['esc10','src_file','take'])

df_urban = df2.drop(columns=['fsID', 'start', 'end', 'salience'])
df_urban = df_urban.rename(columns={'slice_file_name':'filename', 'class':'category', 'classID':'target'})

# Disini dua duanya df udah sama persis

# Keep in mind ada beberapa category tuh yang sama, kyk dog dan dog_bark, ini perlu dibersihin juga tapi ntaran
# Next kita tambahin sumber pada tiap dataframe
df_esc['source'] = "esc"
df_urban['source'] = "urban"

# for fold_iterate in range(6,11):
#     df_urban.loc[df_urban.fold == fold_iterate, 'fold'] = fold_iterate-5

# print(df_esc.head())
# print(df_urban.head())

# Btw dog_bark gw samain dengan dog dlu ya
df_urban.loc[df_urban.category == 'dog_bark', "category"] = "dog"

# Sekarang gabungin 2 dataframenya yuhu
df_combine = pd.concat([df_esc, df_urban])

# Next, tiap hasil category harus kita kasi id buat penanda dan jadi output di model
# Ini list category yang ada
classes = df_combine['category'].unique()
# Ini bikin ID untuk tiap category
class_dict = {i:x for x,i in enumerate(classes)}
#print(class_dict)
#print(len(classes))

# Terus kalo udah pnya dictionarynya, kita taruh nilai ID targetnya ke column 'target'
df_combine['target'] = df_combine['category'].map(class_dict)

df_combine
# Udah kelar nih, tinggal kita preprocessing jadi melspectogram cuy

Yang ini ga ada perubahan

In [ ]:
# Class Conf will save the settings we are going to use in this notebook
class conf:
    sr = 16000
    duration = 3
    hop_length = 340*duration
    fmin = 20
    fmax = sr // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sr * duration
    epochs = 30

def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sr)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sr,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

def show_melspectrogram(conf, mels, title='Log-frequency power spectrogram'):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf.sr, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()

# lanjut ke preprocess


In [ ]:
INPUTSHAPE = (128, 32, 1)
def create_model():
    created_model =  models.Sequential([
        layers.Conv2D(64 , (3,3),activation = 'relu',padding='same', input_shape = INPUTSHAPE),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3,3), activation='relu',padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2), strides=(2,2)),
        layers.Dropout(0.2),

        layers.Conv2D(128, (3,3), activation='relu',padding='same'),                      
        layers.BatchNormalization(),
        layers.Conv2D(128, (3,3), activation='relu',padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2), strides=(2,2)),
        layers.Dropout(0.2),

        layers.Conv2D(256, (3,3), activation='relu',padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3,3), activation='relu',padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2), strides=(2,2)),    
        layers.Dropout(0.2),

        layers.GlobalAveragePooling2D(),

        layers.Dense(256 , activation = 'relu'),
        layers.Dense(256 , activation = 'relu'),
        layers.Dense(len(classes) , activation = 'softmax')
    ])

    created_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])
    return created_model

In [ ]:
# Our model summary
model = create_model()
print(model.summary())
%mkdir "cpkt"
%mkdir "logs"
LOGDIR = "logs"
CPKT = "cpkt/"

#this callback is used to prevent overfitting.
callback_1 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

#this checkpoint saves the best weights of model at every epoch
callback_2 = tf.keras.callbacks.ModelCheckpoint(
    CPKT, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None
)

#this is for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOGDIR)

In [ ]:
x_train , y_train = [] , []
x_val , y_val = [] , []

for data in df_combine[1998:2002].itertuples():
    print(data)
    if(data[5]=='esc'):
        sig , sr = librosa.load(DATA_ESC+data[1], sr=16000)
    else:
        sig , sr = librosa.load(DATA_URBAN + str(data[2]) + '/' + data[1], sr=16000)
    #Creating three random 2 second clip from each audio file, to create more samples
    for i in range(4):
        print(len(sig)/16000)
        sig_ = sig[i : int((i+2)+sr)]
        mel_spec = audio_to_melspectrogram(conf, sig_)
        print(mel_spec.shape)
        x_train.append(mel_spec)
        y_train.append(data[3])


# Bagian sini diubah
Aku kasi If Else disini
Sisanya sama kyk yg sebelumnya

In [ ]:
def preprocess(fold):
    x_train , y_train = [] , []
    x_val , y_val = [] , []
    
    train_df = df_combine[df_combine.fold == fold]
    val_df = df_combine[df_combine.fold == fold]
    
    for data in df_combine[1990:2020].itertuples():
        print(data[0])
        if(data[5]=='esc'):
            sig , sr = librosa.load(DATA_ESC+data[1], sr=16000)
        else:
            sig , sr = librosa.load(DATA_URBAN + str(data[2]) + '/' + data[1], sr=16000)
        #Creating three random 2 second clip from each audio file, to create more samples
        for i in range(4):
            sig_ = sig[i : int((i+2)+sr)]
            mel_spec = audio_to_melspectrogram(conf, sig_)
            x_train.append(mel_spec)
            y_train.append(data[3])

    for data in df_combine[1990:2020].itertuples():
        print(data[0])
        if(data[5]=='esc'):
            sig , sr = librosa.load(DATA_ESC+data[1], sr=16000)
        else:
            sig , sr = librosa.load(DATA_URBAN + str(data[2]) + '/' + data[1], sr=16000)
        #Creating three random 2 second clip from each audio file, to create more samples
        for i in range(4):
            sig_ = sig[i : int((i+2)+sr)]
            mel_spec = audio_to_melspectrogram(conf, sig_)
            x_val.append(mel_spec)
            y_val.append(data[3])
            
    # convert list to numpy array
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    x_val = np.array(x_val)
    y_val = np.array(y_val)

    #one-hot encoding the target
    y_train = tf.keras.utils.to_categorical(y_train , num_classes=len(classes))
    y_val = tf.keras.utils.to_categorical(y_val , num_classes=len(classes))

    # our tensorflow model takes input as (no_of_sample , height , width , channel).
    # here X has dimension (no_of_sample , height , width).
    # So, the below code will reshape it to (no_of_sample , height , width , 1).
    x_train, y_train = shuffle(x_train, y_train)
    x_val, y_val = shuffle(x_val, y_val)
    
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
    x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], 1)
    
    return (x_train, y_train, x_val, y_val)

In [ ]:
#save the model history in a list after fitting so that we can plot later
model_history = []
metrics = []


# The training section will use k-fold cross validation, just as suggested by the dataset creator.
# Cross Validation training will make a more robust model and prevent bias on training.
for fold in range(1, 6):
    print('\n\nTraining fold', fold)
    print('*' * 20)
    
    x_train, y_train, x_val, y_val = preprocess(fold)
    model = create_model()
    history = model.fit(x_train,y_train ,
            validation_data=(x_val,y_val),
            epochs=conf.epochs,
            callbacks = [callback_1], verbose=2)
    eval_score = model.evaluate(x_val, y_val)
    print("Val Score: ",eval_score )
    model_history.append(history)
    metrics.append(eval_score)